<a href="https://colab.research.google.com/github/mmaguero/diploma_fpuna_nlp_ia/blob/master/2025/Gemma_3N_4B_finetuning_with_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Competition](attachment:4ba23f21-a6df-45a7-b3ab-0460b9700288.png)](https://www.kaggle.com/competitions/google-gemma-3n-hackathon/overview)

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Kaggle instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).

## Gemma 3N Notebook collection:
1. Gemma 3N **Multimodal inference + conversational finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-multimodal-finetuning-inference) *⬅️ Your are here*
2. Gemma 3N **Vision finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-vision-finetuning)
3. Gemma 3N **Audio finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-audio-finetuning)

### Installation

In [ ]:
!nvidia-smi

In [2]:
# %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
    !pip install weave
    !pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.11.4 requires msgspec, which is not installed.
unsloth-zoo 2025.11.4 requires tyro, which is not installed.
unsloth-zoo 2025.11.4 requires torchao>=0.13.0, but you have torchao 0.10.0 which is incompatible.
unsloth-zoo 2025.11.4 requires trl!=0.19.

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=2e816b7e1ee3d7e93d0a60be34be9abd41aee4f8ac1dd2ba03fedfdc6bd25a04
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [3]:
#%%capture
# Install latest transformers for Gemma 3N
!pip install --no-deps --upgrade transformers # Only for Gemma 3N
!pip install --no-deps --upgrade timm # Only for Gemma 3N

We encourage you to login to your Hugging Face account so you can upload and share your model with the community. When prompted, enter your token to login:

In [1]:
from huggingface_hub import notebook_login

notebook_login()

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision, Text and Audio models!

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3n-E4B-it-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-it-unsloth-bnb-4bit",
    # Pretrained models
    "unsloth/gemma-3n-E4B-unsloth-bnb-4bit",
    "unsloth/gemma-3n-E2B-unsloth-bnb-4bit",

    # Other Gemma 3 quants
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "google/gemma-3n-E4B-it", # "unsloth/gemma-3n-E4B-it", # Or "unsloth/gemma-3n-E2B-it"
    dtype = None, # None for auto detection
    max_seq_length = 768, # 1024, Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


# Gemma 3N can process Text, Vision and Audio!

Let's first experience how Gemma 3N can handle multimodal inputs. We use Gemma 3N's recommended settings of `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from transformers import TextStreamer
import gc
# Helper function for inference
def do_gemma_3n_inference(model, messages, max_new_tokens = 128):
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True, # Must add for generation
        tokenize = True,
        return_dict = True,
        return_tensors = "pt",
    ).to("cuda")
    _ = model.generate(
        **inputs,
        max_new_tokens = max_new_tokens,
        temperature = 1.0, top_p = 0.95, top_k = 64,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )
    # Cleanup to reduce VRAM usage
    del inputs
    torch.cuda.empty_cache()
    gc.collect()

# Gemma 3N can see images!

<img src="https://files.worldwildlife.org/wwfcmsprod/images/Sloth_Sitting_iStock_3_12_2014/story_full_width/8l7pbjmj29_iStock_000011145477Large_mini__1_.jpg" alt="Alt text" height="256">

In [ ]:
sloth_link = "https://files.worldwildlife.org/wwfcmsprod/images/Sloth_Sitting_iStock_3_12_2014/story_full_width/8l7pbjmj29_iStock_000011145477Large_mini__1_.jpg"

messages = [{
    "role" : "user",
    "content": [
        { "type": "image", "image" : sloth_link },
        { "type": "text",  "text" : "Mba'e pelíkulapepa ojehechauka ko mymba?" }
    ]
}]
# You might have to wait 1 minute for Unsloth's auto compiler
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

Let's make a poem about sloths!

In [ ]:
messages = [{
    "role": "user",
    "content": [{ "type" : "text",
                  "text" : "Ehai peteĩ ñe'ẽpoty perezoso (peteï mymba ojoguáva Mapinguari-pe) peguara: " }]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 128)

# Gemma 3N can also hear!

In [ ]:
from IPython.display import Audio, display
Audio("https://www.nasa.gov/wp-content/uploads/2015/01/591240main_JFKmoonspeech.mp3")

In [ ]:
!wget -qqq https://www.nasa.gov/wp-content/uploads/2015/01/591240main_JFKmoonspeech.mp3 -O audio.mp3

In [ ]:
audio_file = "audio.mp3"

messages = [{
    "role" : "user",
    "content": [
        { "type": "audio", "audio" : audio_file },
        { "type": "text",  "text" : "Mba'érehepa oñe'ẽ ko audio?" }
    ]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

# Let's combine all 3 modalities together!

In [ ]:
messages = [{
    "role" : "user",
    "content": [
        { "type": "audio", "audio" : audio_file },
        { "type": "image", "image" : sloth_link },
        { "type": "text",  "text" : "Mba'érehepa oñe'ẽ ko audio ha ta'anga? "\
                                    "Mba'éichapa ojoaju hikuái?" }
    ]
}]
do_gemma_3n_inference(model, messages, max_new_tokens = 256)

# Let's finetune Gemma 3N!

You can finetune the vision and text parts for now through selection - the audio part can also be finetuned - we're working to make it selectable as well!

We now add LoRA adapters so we only need to update a small amount of parameters!

In [ ]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # Should leave on always!

    r = 4,           # 8, Larger = higher accuracy, but might overfit
    lora_alpha = 4,  # 8, Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 42, #3407,
)

<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

We get the first 3000 rows of the dataset

In [ ]:
from datasets import load_dataset
#dataset = load_dataset("mlabonne/FineTome-100k", split = "train[:3000]")
dataset = load_dataset("rubuntu/dataset-guarani-jopara-v01")

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset_train = standardize_data_formats(dataset["train"])

In [ ]:
dataset_dev = standardize_data_formats(dataset["validation"])

In [ ]:
dataset_test = standardize_data_formats(dataset["test"])

Let's see how row 100 looks like!

In [ ]:
dataset_train[100]

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`. We remove the `<bos>` token using removeprefix(`'<bos>'`) since we're finetuning. The Processor will add this token before training and the model expects only one.

In [ ]:
def formatting_prompts_func(examples):
   convos_1 = examples["instruction"]
   convos_2 = examples["input"]
   convos_3 = examples["output"]

   texts = [
       tokenizer.apply_chat_template(
           [
               {'role': 'user', 'content': [{'type': 'text', 'text': convos_1[i] + " " + convos_2[i]}]},
               {'role': 'assistant', 'content': [{'type': 'text', 'text': convos_3[i]}]},
           ],
           tokenize = False,
           add_generation_prompt = False
       ).removeprefix('<bos>')
       for i in range(len(convos_1))
   ]
   return { "text" : texts, }

dataset_train = dataset_train.map(formatting_prompts_func, batched = True)
dataset_dev = dataset_dev.map(formatting_prompts_func, batched = True)
dataset_test = dataset_test.map(formatting_prompts_func, batched = True)

Let's see how the chat template did! Notice there is no `<bos>` token as the processor tokenizer will be adding one.

In [ ]:
dataset_train[100]["text"]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [ ]:
! pip install evaluate rouge_score

In [ ]:
from transformers import EvalPrediction
import numpy as np
import evaluate # Import the evaluate library for ROUGE and BLEU
# Uncomment the line below if you encounter errors related to NLTK tokenizers
# import nltk
# nltk.download('punkt')

# Load metrics outside the function to avoid re-loading repeatedly
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

def compute_metrics(p: EvalPrediction):
    logits = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    pred_ids = np.argmax(logits, axis=-1)
    label_ids = p.label_ids

    decoded_preds = []
    decoded_labels = []

    for i in range(len(label_ids)):
        # For references (labels):
        # Filter out -100 (masked tokens) to get only the actual target response tokens
        actual_label_tokens = label_ids[i][label_ids[i] != -100]
        if len(actual_label_tokens) > 0:
            decoded_label = tokenizer.decode(actual_label_tokens, skip_special_tokens=True)
            decoded_labels.append(decoded_label)
        else:
            decoded_labels.append("") # Append empty string if no valid labels

        # For predictions:
        # Extract the GENERATED part from pred_ids.
        # We assume that the length of the prompt in the prediction matches the masked part in labels.
        # Find the start of the response part in labels.
        response_start_idx = np.where(label_ids[i] != -100)[0]
        if len(response_start_idx) > 0:
            response_start_idx = response_start_idx[0]
            # Extract the predicted tokens from this point onwards
            predicted_response_tokens = pred_ids[i][response_start_idx:]
            decoded_pred = tokenizer.decode(predicted_response_tokens, skip_special_tokens=True)
            decoded_preds.append(decoded_pred)
        else:
            decoded_preds.append("") # Append empty string if no valid prediction context

    # Filter out empty strings which can cause issues with metric computation
    filtered_decoded_labels = [text for text in decoded_labels if text.strip()]
    filtered_decoded_preds = [text for text in decoded_preds if text.strip()]

    # Handle cases where there are no valid samples for metric computation
    if not filtered_decoded_labels or not filtered_decoded_preds:
        return {
            #"accuracy": 0.0, # Token accuracy based on initial calculation
            "rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0, "rougeLsum": 0.0,
            "bleu": 0.0
        }

    # Token accuracy (original calculation)
    #valid_preds_for_accuracy = pred_ids[label_ids != -100]
    #valid_labels_for_accuracy = label_ids[label_ids != -100]
    #token_accuracy = np.mean(valid_preds_for_accuracy == valid_labels_for_accuracy) if len(valid_labels_for_accuracy) > 0 else 0.0

    # ROUGE
    rouge_results = rouge_metric.compute(predictions=filtered_decoded_preds,
                                         references=filtered_decoded_labels,
                                         use_stemmer=True)

    # BLEU
    bleu_results = bleu_metric.compute(predictions=filtered_decoded_preds,
                                       references=filtered_decoded_labels)

    metrics = {
        #"accuracy": token_accuracy,
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "rougeLsum": rouge_results["rougeLsum"],
        "bleu": bleu_results["bleu"] if bleu_results and "bleu" in bleu_results else 0.0,
    }

    # combined
    metrics["combined"] = (metrics["rouge1"] + metrics["rouge2"] +
                          metrics["rougeL"] + metrics["rougeLsum"] +
                          metrics["bleu"])/5

    return metrics

In [ ]:
from transformers import TrainerCallback
import psutil

# Define the MemoryMonitorCallback class as provided by the user
class MemoryMonitorCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        # Python process RAM usage
        process = psutil.Process(os.getpid())
        python_ram_mb = process.memory_info().rss / (1024 ** 2)

        # GPU VRAM usage
        if torch.cuda.is_available():
            gpu_allocated_mb = torch.cuda.memory_allocated() / (1024 ** 2)
            gpu_max_allocated_mb = torch.cuda.max_memory_allocated() / (1024 ** 2)
            print(f"\nStep {state.global_step}: Python RAM = {python_ram_mb:.2f} MB, GPU Allocated = {gpu_allocated_mb:.2f} MB, GPU Max Allocated = {gpu_max_allocated_mb:.2f} MB")
        else:
            print(f"\nStep {state.global_step}: Python RAM = {python_ram_mb:.2f} MB")

# Instantiate the callback
memory_monitor_callback = MemoryMonitorCallback()

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_dev, # Can set up evaluation!
    compute_metrics = compute_metrics, # Now includes ROUGE and BLEU!
    callbacks=[memory_monitor_callback], # Add the custom callback here
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8, # Use GA to mimic batch size!
        warmup_steps = 5,
        # num_train_epochs = 2, # Set this for full training run.
        max_steps = 1000,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 500,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        report_to = "wandb", # Use this for WandB etc
        do_eval=True,
        eval_strategy="steps",
        eval_steps=500,
        metric_for_best_model="combined",
        greater_is_better=True,
        gradient_checkpointing = True, # New: Enable gradient checkpointing
    ),
)

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Let's verify masking the instruction part is done! Let's print the 100th row again.  Notice how the sample only has a single `<bos>` as expected!

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

Now let's print the masked out example - you should see only the answer is present:

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

In [ ]:
# @title Show current memory stats
import gc
gc.collect()
torch.cuda.empty_cache()
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

# Let's train the model!

To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# @title Show final memory and time stats
gc.collect()
torch.cuda.empty_cache()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "Ehaíve ko secuencia: 1, 1, 2, 3, 5, 8,",
    }]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")
outputs = model.generate(
    **inputs,
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Mba'érepa hovy yvága?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
#model.save_pretrained("gemma-3n")  # Local saving
#tokenizer.save_pretrained("gemma-3n")
model.push_to_hub("mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it") #, token = hf_token) # Online saving
tokenizer.push_to_hub("mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it") #, token = hf_token) # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it", #"lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Mba'épa ha'e pe Gemma-3N?",}]
}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
    tokenize = True,
    return_dict = True,
).to("cuda")

from transformers import TextStreamer
_ = model.generate(
    **inputs,
    max_new_tokens = 128, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3N-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3N-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if True: # Change to True to upload finetune
    model.push_to_hub_merged(
        "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it", tokenizer,
        #token = hf_token
    )

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3N-finetune",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
if True: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "mmaguero/alpaca-3.5-gn-gemma-3n-E4B-it-gguf",
        #token = hf_token,
    )

Now, use the `gemma-3N-finetune.gguf` file or `gemma-3N-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

## Gemma 3N Notebook collection:
1. Gemma 3N **Multimodal inference + conversational finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-multimodal-finetuning-inference) *⬅️ Your are here*
2. Gemma 3N **Vision finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-vision-finetuning)
3. Gemma 3N **Audio finetuning** [Kaggle Notebook](https://www.kaggle.com/code/danielhanchen/gemma-3n-4b-audio-finetuning)

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
